# Task 5.6: Documentation and Benchmarks

**Module:** 5 - Phase 1 Capstone: MicroGrad+  
**Time:** 1 hour  
**Difficulty:** ⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand how to write good API documentation
- [ ] Compare MicroGrad+ performance against PyTorch
- [ ] Learn why PyTorch is faster and what optimizations it uses
- [ ] Complete the Phase 1 Capstone!

---

## 📚 Prerequisites

- Completed: Tasks 5.1-5.5
- Knowledge of: Python documentation practices

In [ ]:
import numpy as np
import time
import sys
from pathlib import Path

def _find_module_root():
    """Find the module root directory containing micrograd_plus."""
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / 'micrograd_plus' / '__init__.py').exists():
            return str(parent)
    # Fallback to parent directory
    return str(Path.cwd().parent)

sys.path.insert(0, _find_module_root())

from micrograd_plus import (
    Tensor, Linear, ReLU, Dropout,
    CrossEntropyLoss, MSELoss, Adam, SGD, Sequential
)
from micrograd_plus.utils import set_seed

set_seed(42)

---

## Part 1: API Documentation

Good documentation makes your code usable by others (and your future self!). Let's review the key components of our MicroGrad+ API.

In [ ]:
# Explore the module documentation
import micrograd_plus

print("MicroGrad+ Module Documentation")
print("=" * 60)
print(micrograd_plus.__doc__)

In [ ]:
# Check individual class documentation
print("Tensor Class Documentation")
print("=" * 60)
print(Tensor.__doc__)

In [ ]:
# Example of good docstring format
print("Linear Layer Documentation")
print("=" * 60)
print(Linear.__doc__)

### Documentation Best Practices

1. **Module docstring**: Explain what the module does and provide a quick example
2. **Class docstring**: Describe purpose, args, attributes, and usage
3. **Method docstring**: Document args, returns, raises, and provide examples
4. **Type hints**: Use them for better IDE support and clarity

---

## Part 2: Benchmarking Against PyTorch

Let's compare MicroGrad+ performance against PyTorch. Spoiler: PyTorch will be much faster - and that's okay! The goal is to understand why.

In [ ]:
# Check if PyTorch is available
try:
    import torch
    import torch.nn as nn
    PYTORCH_AVAILABLE = True
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
except ImportError:
    PYTORCH_AVAILABLE = False
    print("PyTorch not available. Skipping PyTorch benchmarks.")

In [ ]:
def benchmark_micrograd(batch_size, in_features, hidden, out_features, num_iterations):
    """Benchmark MicroGrad+ forward and backward pass."""
    set_seed(42)
    
    # Create model
    model = Sequential(
        Linear(in_features, hidden),
        ReLU(),
        Linear(hidden, hidden),
        ReLU(),
        Linear(hidden, out_features)
    )
    
    loss_fn = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)
    
    # Create data
    X = np.random.randn(batch_size, in_features).astype(np.float32)
    y = np.random.randint(0, out_features, batch_size)
    
    # Warmup
    for _ in range(5):
        x_tensor = Tensor(X, requires_grad=True)
        logits = model(x_tensor)
        loss = loss_fn(logits, Tensor(y))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Benchmark
    start = time.time()
    for _ in range(num_iterations):
        x_tensor = Tensor(X, requires_grad=True)
        logits = model(x_tensor)
        loss = loss_fn(logits, Tensor(y))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    elapsed = time.time() - start
    
    return elapsed / num_iterations

In [ ]:
if PYTORCH_AVAILABLE:
    def benchmark_pytorch(batch_size, in_features, hidden, out_features, num_iterations, device='cpu'):
        """Benchmark PyTorch forward and backward pass."""
        torch.manual_seed(42)
        
        # Create model
        model = nn.Sequential(
            nn.Linear(in_features, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_features)
        ).to(device)
        
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Create data
        X = torch.randn(batch_size, in_features, device=device)
        y = torch.randint(0, out_features, (batch_size,), device=device)
        
        # Warmup
        for _ in range(5):
            logits = model(X)
            loss = loss_fn(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        # Benchmark
        start = time.time()
        for _ in range(num_iterations):
            logits = model(X)
            loss = loss_fn(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        elapsed = time.time() - start
        
        return elapsed / num_iterations

In [ ]:
# Run benchmarks
print("Benchmark: Forward + Backward + Optimizer Step")
print("=" * 60)

configs = [
    (32, 784, 256, 10, 100),   # Small: MNIST-like
    (64, 784, 512, 10, 50),    # Medium
    (128, 1024, 512, 100, 20), # Large
]

results = []

for batch_size, in_features, hidden, out_features, iters in configs:
    print(f"\nConfig: batch={batch_size}, input={in_features}, hidden={hidden}, output={out_features}")
    
    # MicroGrad+
    mg_time = benchmark_micrograd(batch_size, in_features, hidden, out_features, iters)
    print(f"  MicroGrad+: {mg_time*1000:.2f} ms/iteration")
    
    if PYTORCH_AVAILABLE:
        # PyTorch CPU
        pt_cpu_time = benchmark_pytorch(batch_size, in_features, hidden, out_features, iters, 'cpu')
        print(f"  PyTorch CPU: {pt_cpu_time*1000:.2f} ms/iteration")
        
        ratio_cpu = mg_time / pt_cpu_time
        print(f"  Ratio (MicroGrad+ / PyTorch CPU): {ratio_cpu:.1f}x")
        
        # PyTorch GPU (if available)
        if torch.cuda.is_available():
            pt_gpu_time = benchmark_pytorch(batch_size, in_features, hidden, out_features, iters, 'cuda')
            print(f"  PyTorch GPU: {pt_gpu_time*1000:.2f} ms/iteration")
            ratio_gpu = mg_time / pt_gpu_time
            print(f"  Ratio (MicroGrad+ / PyTorch GPU): {ratio_gpu:.1f}x")
        
        results.append({
            'config': f'{batch_size}x{in_features}',
            'micrograd': mg_time * 1000,
            'pytorch_cpu': pt_cpu_time * 1000,
            'ratio': ratio_cpu
        })

In [ ]:
# Visualize benchmark results
if PYTORCH_AVAILABLE and results:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    configs = [r['config'] for r in results]
    mg_times = [r['micrograd'] for r in results]
    pt_times = [r['pytorch_cpu'] for r in results]
    ratios = [r['ratio'] for r in results]
    
    # Time comparison
    x = np.arange(len(configs))
    width = 0.35
    
    axes[0].bar(x - width/2, mg_times, width, label='MicroGrad+', color='blue')
    axes[0].bar(x + width/2, pt_times, width, label='PyTorch CPU', color='orange')
    axes[0].set_ylabel('Time (ms)')
    axes[0].set_title('Execution Time Comparison')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(configs)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Ratio
    axes[1].bar(configs, ratios, color='green')
    axes[1].set_ylabel('Slowdown Factor (x times slower)')
    axes[1].set_title('MicroGrad+ vs PyTorch CPU')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

## Part 3: Why is PyTorch Faster?

PyTorch is typically 10-100x faster than our implementation. Here's why:

### 1. Compiled Operations (C++/CUDA)
```
MicroGrad+: Pure Python + NumPy
PyTorch: C++ kernels, CUDA for GPU
```

### 2. Optimized Memory Layout
```
MicroGrad+: NumPy arrays (general purpose)
PyTorch: Custom tensor layout optimized for ML
```

### 3. Operator Fusion
```
MicroGrad+: Each operation creates new array
PyTorch: Fuses operations to reduce memory traffic
```

### 4. Parallelization
```
MicroGrad+: Single-threaded (mostly)
PyTorch: Multi-threaded CPU, massively parallel GPU
```

### 5. Autograd Optimization
```
MicroGrad+: Python closures for backward functions
PyTorch: Compiled autograd engine with memory optimization
```

In [ ]:
# Demonstrate why GPU is so fast
print("Why GPUs are so fast for neural networks:")
print("="*60)
print("""
Matrix multiplication example: (1000 x 1000) @ (1000 x 1000)

Operations needed: 1000 * 1000 * 1000 = 1 billion multiply-adds

CPU (8 cores, 3 GHz):
  - ~24 billion ops/sec
  - Time: ~40 ms

GPU (6144 CUDA cores on DGX Spark):
  - ~200+ TFLOPS (FP8/FP16)
  - Time: ~5 μs (microseconds!)

The GPU wins because:
1. Thousands of simple cores vs dozens of complex cores
2. High memory bandwidth (designed for throughput)
3. Operations optimized for data parallelism
""")

---

## Part 4: What You've Accomplished

Let's summarize what you built in this capstone project!

In [ ]:
# Count lines of code in MicroGrad+
import os

def count_lines(filepath):
    """Count non-empty, non-comment lines."""
    count = 0
    with open(filepath, 'r') as f:
        for line in f:
            stripped = line.strip()
            if stripped and not stripped.startswith('#'):
                count += 1
    return count

package_dir = '../micrograd_plus'
total_lines = 0

print("MicroGrad+ Package Statistics")
print("="*60)

for filename in os.listdir(package_dir):
    if filename.endswith('.py'):
        filepath = os.path.join(package_dir, filename)
        lines = count_lines(filepath)
        total_lines += lines
        print(f"  {filename:20s}: {lines:4d} lines")

print(f"  {'TOTAL':20s}: {total_lines:4d} lines")

In [ ]:
# Summary of what was built
print("""
🎉 CONGRATULATIONS! You've completed the Phase 1 Capstone! 🎉

You built MicroGrad+, featuring:

✅ Tensor Class with Autograd
   - Automatic differentiation (reverse-mode)
   - Broadcasting support
   - 15+ differentiable operations

✅ Neural Network Layers
   - Linear (fully connected)
   - ReLU, Sigmoid, Tanh activations
   - Softmax for classification
   - Dropout for regularization
   - BatchNorm for normalization

✅ Loss Functions
   - MSE Loss for regression
   - Cross-Entropy Loss for classification

✅ Optimizers
   - SGD with momentum
   - Adam (adaptive learning rates)

✅ Training Utilities
   - Sequential container
   - DataLoader for batching
   - Training and evaluation loops

✅ Testing Suite
   - Unit tests for all components
   - Gradient checking against numerical gradients

✅ Real Application
   - Trained on MNIST with >95% accuracy!
""")

---

## Part 5: What's Next?

In **Phase 2: Intermediate**, you'll use PyTorch to:

1. **Work with real hardware**
   - Use the DGX Spark GPU
   - Leverage 128GB unified memory
   - See 10-100x speedups!

2. **Build advanced architectures**
   - Convolutional Neural Networks (CNNs)
   - Recurrent Neural Networks (RNNs)
   - Transformers and Attention

3. **Handle real-world data**
   - Image classification (CIFAR-10, ImageNet)
   - Natural Language Processing
   - Transfer learning

4. **Scale to larger models**
   - Fine-tuning pre-trained models
   - Efficient training techniques
   - Model optimization

---

## 🎉 Phase 1 Complete!

You now have:
- Deep understanding of how neural networks work internally
- Your own working autograd engine
- Solid foundation in Python, NumPy, and software engineering
- Practical experience training real models

These fundamentals will serve you well as you move to more advanced topics!

---

## 🧹 Cleanup

In [ ]:
import gc
gc.collect()
print("Phase 1 Capstone Complete! 🎓")